In [ ]:
!pip install -q datasets
!pip install -q bitsandbytes
!pip install -q peft
!pip install -q accelerate
!pip install -q trl
!pip install -q wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.1/280.1 kB 11

In [ ]:
# !pip install --upgrade pyarrow

In [ ]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# General parameters
model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"  # The model that you want to train from the Hugging Face hub
new_model = "llamma-prescription-json-generator"  # The name for fine-tuned LoRA Adaptor

In [ ]:
# LoRA parameters
lora_r = 32
lora_alpha = lora_r * 2
lora_dropout = 0.1
target_modules = ["q_proj", "v_proj", "k_proj"]

In [ ]:
# QLoRA parameters
load_in_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
bnb_4bit_use_double_quant = False

In [ ]:
# TrainingArguments parameters
num_train_epochs = 10
fp16 = False
bf16 = False
per_device_train_batch_size = 4
gradient_accumulation_steps = 1
gradient_checkpointing = True
learning_rate = 0.00015
weight_decay = 0.01
optim = "paged_adamw_32bit"
lr_scheduler_type = "cosine"
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 25
logging_steps = 10

# SFT parameters
max_seq_length = None
packing = False
device_map = {"": 0}

# Dataset parameters
use_special_template = True
response_template = " ### Answer:"
instruction_prompt_template = '"### Human:"'
use_llama_like_model = True

In [ ]:
def load_data(file_path):
    percent_of_train_dataset = 0.90
    data = load_dataset("json", data_files=file_path, split="train")

    split_dataset = data.train_test_split(
        train_size=int(data.num_rows * percent_of_train_dataset), seed=19, shuffle=False
    )
    train_dataset = split_dataset["train"]
    eval_dataset = split_dataset["test"]
    print(f"Size of the train set: {len(train_dataset)}. Size of the validation set: {len(eval_dataset)}")
    return train_dataset, eval_dataset

dataset_dict = {"seizure": {}, "prescription": {}}
dataset_dict["prescription"]["train_dataset"], dataset_dict["prescription"]["eval_dataset"] = load_data(
    file_path="/content/prescription.jsonl")

# dataset_dict["seizure"]["train_dataset"], dataset_dict["seizure"]["eval_dataset"] = load_data(
#     file_path="/content/seizure_frequency.jsonl")

Generating train split: 0 examples [00:00, ? examples/s]

Size of the train set: 264. Size of the validation set: 30


In [ ]:
# Load LoRA configuration
peft_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=target_modules,
)

In [ ]:
# Load QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=load_in_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=bnb_4bit_use_double_quant,
)

In [ ]:
# Load base model
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             quantization_config=bnb_config,
                                             device_map=device_map)
model.config.use_cache = False

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [ ]:
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=new_model,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    gradient_checkpointing=gradient_checkpointing,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    do_eval=True,
    evaluation_strategy="steps"
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"  # Fix weird overflow issue with fp16 training
if not tokenizer.chat_template:
    tokenizer.chat_template = "{% for message in messages %}{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% endfor %}"

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
def special_formatting_prompts(example):
    output_texts = []
    for i in range(len(example["instruction"])):
        text = f"{instruction_prompt_template}{example['instruction'][i]}\n{response_template} {example['output'][i]}"
        output_texts.append(text)
    return output_texts


def normal_formatting_prompts(example):
    output_texts = []
    for i in range(len(example["instruction"])):
        chat_temp = [
            {"role": "system", "content": example["instruction"][i]},
            {"role": "assistant", "content": example["output"][i]},
        ]
        text = tokenizer.apply_chat_template(chat_temp, tokenize=False)
        output_texts.append(text)
    return output_texts

In [ ]:
if use_special_template:
    formatting_func = special_formatting_prompts
    if use_llama_like_model:
        response_template_ids = tokenizer.encode(response_template, add_special_tokens=False)[2:]
        collator = DataCollatorForCompletionOnlyLM(response_template=response_template_ids, tokenizer=tokenizer)
    else:
        collator = DataCollatorForCompletionOnlyLM(response_template=response_template, tokenizer=tokenizer)
else:
    formatting_func = normal_formatting_prompts

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_dict["prescription"]["train_dataset"],
    eval_dataset=dataset_dict["prescription"]["eval_dataset"],
    peft_config=peft_config,
    formatting_func=formatting_func,
    data_collator=collator,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/264 [00:00<?, ? examples/s]

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

In [ ]:
if torch.cuda.device_count() > 1:
  model.is_parallelizable = True
  model.model_parallel = True

# Train model
trainer.train()

# Save fine tuned Lora Adaptor
trainer.model.save_pretrained(new_model)

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss,Validation Loss
10,0.778200,0.426850
20,0.285400,0.258787
30,0.287500,0.237321
40,0.393900,0.231038
50,0.201300,0.229113
60,0.202800,0.221055
70,0.370700,0.231590
80,0.171200,0.221799
90,0.315200,0.229006
100,0.192900,0.207389


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an except

In [ ]:
# import torch
# import gc


# def clear_hardwares():
#     torch.clear_autocast_cache()
#     torch.cuda.ipc_collect()
#     torch.cuda.empty_cache()
#     gc.collect()


# clear_hardwares()
# clear_hardwares()

In [ ]:
def generate(model, prompt: str, kwargs):
    tokenized_prompt = tokenizer(prompt, return_tensors="pt").to(model.device)

    prompt_length = len(tokenized_prompt.get("input_ids")[0])

    with torch.cuda.amp.autocast():
        output_tokens = model.generate(**tokenized_prompt, **kwargs) if kwargs else model.generate(**tokenized_prompt)
        output = tokenizer.decode(output_tokens[0][prompt_length:], skip_special_tokens=True)

    return output

In [ ]:
new_model

'llamma-prescription-json-generator'

In [ ]:
new_model_path = "/content/llamma-prescription-json-generator"

In [ ]:
ft_model = AutoModelForCausalLM.from_pretrained(new_model_path)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
peft_model = PeftModel.from_pretrained(ft_model, new_model)
# del base_model

In [ ]:
sample = dataset_dict["prescription"]["eval_dataset"][8]
if use_special_template:
    prompt = f"{instruction_prompt_template}{sample['instruction']}\n{response_template}"
else:
    chat_temp = [{"role": "system", "content": sample["instruction"]}]
    prompt = tokenizer.apply_chat_template(chat_temp, tokenize=False,
                                           add_generation_prompt=True)

prompt

'"### Human:"Your task is to extract prescription information from a clinical text\n\nBelow is the clinical notes from a doctor, delimited by triple quotes.\nclinical text: ```Clinic date 25/09/2018  Re Mr Owen Owens30/06/1968  Dear doctor,  Diagnosis: Symptomatic structural focal epilepsy Area of ischaemic damage left inferior frontal lobe Significant anxiety and depression  I reviewed this 50 year old man, together with his wife, in clinic today. He was well from an epilepsy point of view until he had a   seizur  e last month. His wife heard a bang from the next room and went in to see him unconsciouss on the floor. He was stiff, his eyes were open and he made unusual groaning noises before starting to shake. This lasted about 40 seconds before he went to a “deep sleep”. It took him several hours to recover fully. Mr Owens himself remembers his right leg twitching before he lost consciousness. I think therefore that this was a   focal to bilateral convulsive   seizur    es. As you re

In [ ]:
gen_kwargs = {"max_new_tokens": 100}
generated_texts = generate(model=peft_model, prompt=prompt, kwargs=gen_kwargs)
print(generated_texts)

<ipython-input-20-cac47dd2b0e1>:6: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


 {'start_index': '917', 'end_index': '950', 'text': 'He is currently taking lamotrigine 75mg twice a day', 'attributes': {'DrugName': 'lamotrigine', 'DrugDose': '75', 'DoseUnit':'mg', 'Frequency': '2'}} {'start_index': '965', 'end_index': '988', 'text': 'He is currently taking lamotrigine 75mg twice a day


In [ ]:
from tqdm.notebook import tqdm

In [ ]:
progress_bar = tqdm(total=len(dataset_dict["prescription"]["eval_dataset"]))
generated_texts_list = []
for eval_sample in dataset_dict["prescription"]["eval_dataset"]:
  chat_temp = [{"role": "system", "content": eval_sample["instruction"]}]
  prompt = tokenizer.apply_chat_template(chat_temp, tokenize=False,
                                           add_generation_prompt=True)
  generated_texts_list.append(generate(model=peft_model, prompt=prompt,
                                       kwargs=gen_kwargs))
  progress_bar.update(1)
progress_bar.close()

  0%|          | 0/30 [00:00<?, ?it/s]

<ipython-input-20-cac47dd2b0e1>:6: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setti

In [ ]:
len(generated_texts_list)

30

In [ ]:
import pandas as pd

In [ ]:
tmp = dataset_dict["prescription"]["eval_dataset"].to_dict()
tmp = pd.DataFrame.from_dict(tmp)

tmp.head()

,instruction,output
0,Your task is to extract prescription informati...,"{'start_index': '547', 'end_index': '558', 'te..."
1,Your task is to extract prescription informati...,"{'start_index': '802', 'end_index': '858', 'te..."
2,Your task is to extract prescription informati...,"{'start_index': '155', 'end_index': '187', 'te..."
3,Your task is to extract prescription informati...,"{'start_index': '224', 'end_index': '241', 'te..."
4,Your task is to extract prescription informati...,"{'start_index': '224', 'end_index': '240', 'te..."


In [ ]:
tmp["generated_text"] = generated_texts_list

tmp.head()

,instruction,output,generated_text
0,Your task is to extract prescription informati...,"{'start_index': '547', 'end_index': '558', 'te...","{'start_index': '514', 'end_index': '525', 'te..."
1,Your task is to extract prescription informati...,"{'start_index': '802', 'end_index': '858', 'te...","{'start_index': '256', 'end_index': '275', 'te..."
2,Your task is to extract prescription informati...,"{'start_index': '155', 'end_index': '187', 'te...","{'start_index': '214', 'end_index': '236', 'te..."
3,Your task is to extract prescription informati...,"{'start_index': '224', 'end_index': '241', 'te...","{'start_index': '204', 'end_index': '217', 'te..."
4,Your task is to extract prescription informati...,"{'start_index': '224', 'end_index': '240', 'te...","{'start_index': '240', 'end_index': '260', 'te..."


In [ ]:
tmp.to_csv("prescription_output.csv")

In [ ]:
prompt2 = f"Please validate the given json  {generated_texts} please extract and respond only the first one that is a complete dictionary in the json format"
#generated_texts = generate(model=model, prompt=prompt2, kwargs=gen_kwargs)
# print(generated_texts)

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model="google/gemma-2-2b",
    device="cuda",  # replace with "mps" to run on a Mac device
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
outputs = pipe(prompt2, max_new_tokens=100)
response = outputs[0]["generated_text"]
print(response)

In [ ]:
sample["output"]

{'entity': 'Prescription',
 'start_index': '547',
 'end_index': '558',
 'text': 'lamotrigine',
 'attributes': {'DrugName': 'Lamotrigine',
  'DrugDose': '150',
  'DoseUnit': 'mg',
  'Frequency': '2'}}

In [ ]:
from jsonformer import Jsonformer

In [ ]:
prompt

'"### Human:"Here is the clinical text from the doctor in the delimiter. clinical text: <<<Clinic date 25/09/2018  Re Mr Owen Owens30/06/1968  Dear doctor,  Diagnosis: Symptomatic structural focal epilepsy Area of ischaemic damage left inferior frontal lobe Significant anxiety and depression  I reviewed this 50 year old man, together with his wife, in clinic today. He was well from an epilepsy point of view until he had a   seizur  e last month. His wife heard a bang from the next room and went in to see him unconsciouss on the floor. He was stiff, his eyes were open and he made unusual groaning noises before starting to shake. This lasted about 40 seconds before he went to a “deep sleep”. It took him several hours to recover fully. Mr Owens himself remembers his right leg twitching before he lost consciousness. I think therefore that this was a   focal to bilateral convulsive   seizur    es. As you recall Mr Owens has had focal motor   seizur  es in the past where he has had jerking o

In [ ]:
# json_schema = {
#     "type": "object",
#     "properties": {
#         "name": {"type": "string"},
#         "age": {"type": "number"},
#         "is_student": {"type": "boolean"},
#         "courses": {
#             "type": "array",
#             "items": {"type": "string"}
#         }
#     }
# }

json_schema = {
    "type": "object",
    "properties": {
        'entity':  {"type": "string"},
        'start_index': {"type": "number"},
        'end_index': {"type": "number"},
        'text': {"type": "string"},
        'attributes': {
            "type": "object",
            'properties': {
                'DoseUnit': {"type": "string"},
                'Frequency': {"type": "number"},
                'DrugDose': {"type": "string"},
                'DrugName': {"type": "string"}
            }
          }
        }
    }

prompt = "Generate a prescription information based on the following schema:"
jsonformer = Jsonformer(model, tokenizer, json_schema, prompt)
generated_data = jsonformer()

print(generated_data)

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


RuntimeError: The expanded size of the tensor (32000) must match the existing size (32001) at non-singleton dimension 1.  Target sizes: [1, 32000].  Tensor sizes: [32001]